In [2]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
#jan = spark.read.csv('yellow_tripdata_2017-01.csv', header=True, inferSchema = True)
#feb = spark.read.csv('yellow_tripdata_2017-02.csv', header=True, inferSchema = True)
jan = spark.read.csv('Jan17.csv', header=True, inferSchema = True)
feb = spark.read.csv('Feb17.csv', header=True, inferSchema = True)
display(jan)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-1-6e15a40c594c>:4 

In [3]:
vertexP = jan.select("PULocationID").distinct()
vertexP = vertexP.selectExpr("PULocationID as id")
print(vertexP.count())
vertexD = jan.select("DOLocationID").distinct()
vertexD = vertexD.selectExpr("DOLocationID as id")
print(vertexD.count())
if vertexD.count() > vertexP.count():
  vertex = vertexD.unionAll(vertexP)
else : vertex = vertexP.unionAll(vertexD)
print(vertex.count())
vertex = vertex.select("id").distinct()
print(vertex.count())
vertex.show()

112
143
255
157
+---+
| id|
+---+
|148|
|243|
|137|
| 65|
|255|
|133|
| 78|
|211|
|193|
|115|
|210|
| 26|
|192|
|159|
|236|
| 12|
|223|
| 91|
|209|
|128|
+---+
only showing top 20 rows



In [4]:
Edge = jan.select('PULocationID', 'DOLocationID')
Edge.groupBy('PULocationID', 'DOLocationID')
Edge = Edge.selectExpr("PULocationID as src", "DOLocationID as dst")
edge = jan.groupBy('PULocationID', 'DOLocationID').count()
edge = edge.selectExpr("PULocationID as src", "DOLocationID as dst", "count as weight")
edge.show()

+---+---+------+
|src|dst|weight|
+---+---+------+
| 25| 61|     1|
|236|238|     3|
| 49| 49|     1|
|163|263|     1|
|229| 36|     1|
|162| 80|     1|
| 43|  7|     1|
|249|107|     2|
|170|140|     1|
|238|120|     1|
|148|249|     1|
|230|162|     1|
| 68| 74|     1|
|158|234|     1|
|132|132|     1|
|158| 87|     1|
| 48| 75|     2|
|239|234|     1|
| 45| 71|     1|
|129| 48|     1|
+---+---+------+
only showing top 20 rows



In [5]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from graphframes import * 

graph = GraphFrame(vertex,Edge)
vertex.cache()
Edge.cache()

Py4JJavaError: An error occurred while calling o85.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.T

In [6]:
spark.stop()

In [8]:
loc = spark.read.csv('taxi _zone_lookup.csv', header=True, inferSchema = True)

jan = jan.drop('VendorID','store_and_fwd_flag' ,'improvement_surcharge')

In [12]:
import pyspark.sql.functions
split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))
display(jan)

DataFrame[tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: int, trip_distance: double, RatecodeID: int, PULocationID: int, DOLocationID: int, payment_type: int, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, total_amount: double, PUDate: string, PUTime: string, DODate: string, DOTime: string]

In [13]:
jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')
jan.show()

+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+------------+--------+------+--------+------+
|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount|  PUDate|PUTime|  DODate|DOTime|
+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+------------+--------+------+--------+------+
|              1|          3.3|         1|         263|         161|           1|       12.5|  0.0|    0.5|       2.0|         0.0|        15.3|09-01-17| 11:13|09-01-17| 11:25|
|              1|          0.9|         1|         186|         234|           1|        5.0|  0.0|    0.5|      1.45|         0.0|        7.25|09-01-17| 11:32|09-01-17| 11:36|
|              1|          1.1|         1|         164|         161|           1|        5.5|  0.0|    0.5|       1

In [14]:
df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+--------+------+--------+------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|  PUDate|PUTime|  DODate|DOTime|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+--------+------+--------+------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|      Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|09-01-17| 11:13|09-01-17| 11:25|
|    186|Manhattan|Penn Station/Madi...|    234|Manhattan|            Union Sq|              1|     0.9|         1|  186|  234|           1| 5.0|  0

In [15]:
byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byTotalFare.show()

+--------------------+--------------------+---------------+
|              PUZone|              DOZone|avg(total_fare)|
+--------------------+--------------------+---------------+
|        Midtown East|Meatpacking/West ...|          17.75|
|           Sunnyside|         Old Astoria|           12.8|
|            Gramercy|            Flatiron|            7.3|
|        East Village|         JFK Airport|           62.8|
|     Jackson Heights|        Clinton East|           21.8|
|   East Harlem North|            Union Sq|           22.8|
| Little Italy/NoLiTa|         Murray Hill|           15.3|
|Penn Station/Madi...|        Clinton East|           26.3|
|        Clinton East|   East Harlem South|          14.55|
|         JFK Airport|      Bushwick North|           40.3|
|        East Village|    Sunset Park West|           27.8|
|       Midtown South|   East Harlem South|          15.33|
|Upper East Side S...|Upper West Side N...|          15.38|
|   East Harlem North|     Lenox Hill We

In [16]:
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')
byDistance.show()

+--------------------+--------------------+-------------+
|              PUZone|              DOZone|max(distance)|
+--------------------+--------------------+-------------+
|        Midtown East|Meatpacking/West ...|          2.9|
|           Sunnyside|         Old Astoria|         2.94|
|            Gramercy|            Flatiron|          1.0|
|        East Village|         JFK Airport|         17.7|
|     Jackson Heights|        Clinton East|          6.2|
|   East Harlem North|            Union Sq|         6.28|
| Little Italy/NoLiTa|         Murray Hill|         2.82|
|Penn Station/Madi...|        Clinton East|         2.36|
|        Clinton East|   East Harlem South|          3.9|
|         JFK Airport|      Bushwick North|        14.49|
|        East Village|    Sunset Park West|         8.92|
|       Midtown South|   East Harlem South|          4.0|
|Upper East Side S...|Upper West Side N...|         3.13|
|   East Harlem North|     Lenox Hill West|          3.0|
|        Midto

In [7]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
jan = spark.read.csv('../bigdata/yellow_tripdata_2017-01.csv', header=True, inferSchema = True)
jan.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2017-01-09 11:13:28|  2017-01-09 11:25:45|              1|          3.3|         1|                 N|         263|         161|           1|       12.5|  0.0|    0.5|       2.0|         0.0|                  0.3|        15.3|
|       1| 2017-01-09 11:32:27|  2017-01-09 11:3

In [9]:
loc = spark.read.csv('taxi _zone_lookup.csv', header=True, inferSchema = True)

In [10]:
import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(1))

Graph = df_joined.select('PUZone', 'DOZone', 'month')
Graph = Graph.groupBy('PUZone', 'DOZone').count()
Graph = Graph.withColumn("month",lit(1))
Graph = Graph.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Graph = Graph.join(byTotalFare,[Graph.src==byTotalFare.PUZone, Graph.dst==byTotalFare.DOZone],'left_outer')
Graph = Graph.drop('PUZone', 'DOZone')
Graph = Graph.withColumnRenamed("avg(total_fare)", "avg_fare")

Graph.show()

+--------------------+--------------------+-----+-----+------------------+
|                 src|                 dst|count|month|          avg_fare|
+--------------------+--------------------+-----+-----+------------------+
|Allerton/Pelham G...|        East Tremont|    1|    1|              11.8|
|        Battery Park|         Fort Greene|   10|    1|20.003000000000004|
|        Battery Park|     Windsor Terrace|    4|    1|            30.445|
|   Battery Park City|        East Village| 1716|    1| 18.92319930069934|
|        Bloomingdale|           Chinatown|    7|    1| 32.61714285714286|
|        Bloomingdale|     Mariners Harbor|    1|    1|             85.42|
|           Bronxdale|         Eastchester|    1|    1|               9.3|
|  Brooklyn Navy Yard|          Kensington|    1|    1|              18.3|
|      Bushwick North|    Hamilton Heights|    4|    1|           53.8475|
|      Bushwick South|Long Island City/...|    5|    1|            22.432|
|     Carroll Gardens|Upp

In [12]:
########## max distance
max_dist = df_joined.orderBy('distance', ascending=False)
max_dist = max_dist.limit(10)
max_dist = max_dist.withColumnRenamed("distance", "max_distance")
max_dist = max_dist.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist.show()

+--------------------+--------------------+------------+-----+
|              PUZone|              DOZone|max_distance|month|
+--------------------+--------------------+------------+-----+
|Springfield Garde...|                  NA|      264.71|    1|
|          Ocean Hill|Two Bridges/Sewar...|       180.7|    1|
|         JFK Airport| UN/Turtle Bay South|       173.2|    1|
|     Mariners Harbor|                  NA|      156.17|    1|
|        Clinton East|                  NA|       151.7|    1|
|Flushing Meadows-...|                  NA|      143.45|    1|
|                  NA|                  NA|       143.4|    1|
|         JFK Airport|                  NA|       143.4|    1|
|       Richmond Hill|                  NA|       141.8|    1|
|         JFK Airport|                  NA|      139.17|    1|
+--------------------+--------------------+------------+-----+



In [13]:
######max fare
max_fare = df_joined.orderBy('total_fare', ascending=False)
max_fare = max_fare.limit(10)
max_fare = max_fare.withColumnRenamed("total_fare", "max_fare")
max_fare = max_fare.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare.show()

+--------------------+--------------------+--------+-----+
|              PUZone|              DOZone|max_fare|month|
+--------------------+--------------------+--------+-----+
|Upper West Side S...|                  NV|625901.6|    1|
|Two Bridges/Sewar...|Washington Height...|538580.0|    1|
|Upper East Side S...|     Lenox Hill West|  9001.3|    1|
|                  NV|                  NV| 8043.84|    1|
|       Midtown North|Times Sq/Theatre ...|  3009.8|    1|
|                  NV|                  NV|  3009.3|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 1000.29|    1|
+--------------------+--------------------+--------+-----+



In [14]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))

In [15]:
###### by time
PUByTime = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTime = PUByTime.orderBy('count', ascending=False)

In [17]:
####### best PU locs
best_locs = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.orderBy('count', ascending=False)
best_locs.show()

+--------------------+-----+------+
|              PUZone|month| count|
+--------------------+-----+------+
|Upper East Side S...|    1|380663|
|Upper East Side N...|    1|363649|
|      Midtown Center|    1|360995|
|Penn Station/Madi...|    1|339970|
|Times Sq/Theatre ...|    1|334079|
|            Union Sq|    1|333035|
|        Midtown East|    1|330946|
|         Murray Hill|    1|306244|
|        East Village|    1|303580|
|        Clinton East|    1|297414|
| Lincoln Square East|    1|277230|
|Upper West Side S...|    1|252264|
|       Midtown North|    1|250939|
|            Gramercy|    1|238695|
|       Midtown South|    1|227768|
|         JFK Airport|    1|227167|
|     Lenox Hill West|    1|226739|
|        East Chelsea|    1|224558|
|   LaGuardia Airport|    1|212193|
|        West Village|    1|211629|
+--------------------+-----+------+
only showing top 20 rows



In [18]:
#####################################################  Feb

jan = spark.read.csv('../bigdata/yellow_tripdata_2017-02.csv', header=True, inferSchema = True)
jan = jan.drop('VendorID','store_and_fwd_flag' ,'improvement_surcharge')

import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(2))

Grapht = df_joined.select('PUZone', 'DOZone', 'month')
Grapht = Grapht.groupBy('PUZone', 'DOZone').count()
Grapht = Grapht.withColumn("month",lit(2))
Grapht = Grapht.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Grapht = Grapht.join(byTotalFare,[Grapht.src==byTotalFare.PUZone, Grapht.dst==byTotalFare.DOZone],'left_outer')
Grapht = Grapht.drop('PUZone', 'DOZone')
Grapht = Grapht.withColumnRenamed("avg(total_fare)", "avg_fare")
Graph = Graph.union(Grapht)

########## max distance
max_distt = df_joined.orderBy('distance', ascending=False)
max_distt = max_distt.limit(10)
max_distt = max_distt.withColumnRenamed("distance", "max_distance")
max_distt = max_distt.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist = max_dist.union(max_distt)

######max fare
max_faret = df_joined.orderBy('total_fare', ascending=False)
max_faret = max_faret.limit(10)
max_faret = max_faret.withColumnRenamed("total_fare", "max_fare")
max_faret = max_faret.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare = max_fare.union(max_faret)

#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))

###### by time
PUByTimet = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTimet = PUByTimet.orderBy('count', ascending=False)
PUByTime = PUByTime.union(PUByTimet)

####### best PU locs
best_locst = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.union(best_locst)

In [27]:
import pandas as pd
Graph.toPandas().to_csv('mycsv.csv', index=False)

In [30]:
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|   PUHour|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+
|     87|Manhattan|Financial Distric...|    246|Manhattan|West Chelsea/Huds...|              1|    3.29|         1|   87|  246|           1|14.0|  0.0|    0.5| 1.5| 0.0|      16.3|2017-02-03|02:03:50|2017-02-03|02:20:55|    2| midnight|
|     68|Manhattan|        East Chelsea|    239|Manh

In [32]:
### Clustering

from pyspark.ml.feature import VectorAssembler

#vecAssembler = VectorAssembler(inputCols=["PULocID", "PUBorough", "PUZone", "DOLocID","DOBorough", "DOZone", "passenger_count" ,"distance" , "RateCodeID", "srcID", "dstID", "payment_type", "fare" ,"extra", "mta_tax",  "tip","toll","total_fare", "PUDate", "PUTime","DODate","DOTime","month" ,"PUHour"], outputCol="features")
vecAssembler = VectorAssembler(inputCols=["passenger_count" ,"distance" , "RateCodeID", "srcID", "dstID", "payment_type", "fare" ,"extra", "mta_tax","tip","toll","total_fare","month"], outputCol="features")
new_df = vecAssembler.transform(df_joined)
new_df.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+--------------------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|   PUHour|            features|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+--------------------+
|     87|Manhattan|Financial Distric...|    246|Manhattan|West Chelsea/Huds...|              1|    3.29|         1|   87|  246|           1|14.0|  0.0|    0.5| 1.5| 0.0|      16.3|2017-02-03|02:03:50|2017-02-03|02:20:55|    2|

In [33]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=3, seed=1)  # 2 clusters here
model = kmeans.fit(new_df.select('features'))

In [34]:
transformed = model.transform(new_df)
transformed.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+--------------------+----------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|   PUHour|            features|prediction|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+--------------------+----------+
|     87|Manhattan|Financial Distric...|    246|Manhattan|West Chelsea/Huds...|              1|    3.29|         1|   87|  246|           1|14.0|  0.0|    0.5| 1.5| 0.0|      16.3|2017-02-03|02